In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Lactose_Intolerance"

# Input paths
tcga_root_dir = "../../input/TCGA"

# Output paths
out_data_file = "../../output/preprocess/Lactose_Intolerance/TCGA.csv"
out_gene_data_file = "../../output/preprocess/Lactose_Intolerance/gene_data/TCGA.csv"
out_clinical_data_file = "../../output/preprocess/Lactose_Intolerance/clinical_data/TCGA.csv"
json_path = "../../output/preprocess/Lactose_Intolerance/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
import os
import pandas as pd

# 1. List all subdirectories in the TCGA root directory
subdirectories = os.listdir(tcga_root_dir)
print(f"Available TCGA subdirectories: {subdirectories}")

# The target trait is Lactose_Intolerance
target_trait = trait.lower()  # "lactose_intolerance"

# Search for a directory matching our trait (digestive system related)
best_match = None
relevant_keywords = ["digest", "colon", "intestin", "gut", "stomach", "gastro", "coad", "read", "rect", "pancrea"]

for subdir in subdirectories:
    if not os.path.isdir(os.path.join(tcga_root_dir, subdir)) or subdir.startswith('.'):
        continue
        
    subdir_lower = subdir.lower()
    
    # Check if the directory name contains any of our relevant keywords
    for keyword in relevant_keywords:
        if keyword in subdir_lower:
            best_match = subdir
            print(f"Found potential match: {subdir} (matched keyword: {keyword})")
            break
    
    if best_match:
        break

# Handle the case where a match is found
if best_match:
    print(f"Selected directory: {best_match}")
    
    # 2. Get the clinical and genetic data file paths
    cohort_dir = os.path.join(tcga_root_dir, best_match)
    clinical_file_path, genetic_file_path = tcga_get_relevant_filepaths(cohort_dir)
    
    print(f"Clinical file: {os.path.basename(clinical_file_path)}")
    print(f"Genetic file: {os.path.basename(genetic_file_path)}")
    
    # 3. Load the data files
    clinical_df = pd.read_csv(clinical_file_path, sep='\t', index_col=0)
    genetic_df = pd.read_csv(genetic_file_path, sep='\t', index_col=0)
    
    # 4. Print clinical data columns for inspection
    print("\nClinical data columns:")
    print(clinical_df.columns.tolist())
    
    # Print basic information about the datasets
    print(f"\nClinical data shape: {clinical_df.shape}")
    print(f"Genetic data shape: {genetic_df.shape}")
    
    # Check if we have both gene and trait data
    is_gene_available = genetic_df.shape[0] > 0
    is_trait_available = clinical_df.shape[0] > 0
    
else:
    print(f"No suitable directory found for {trait}.")
    is_gene_available = False
    is_trait_available = False

# Record the data availability
validate_and_save_cohort_info(
    is_final=False,
    cohort="TCGA",
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# Exit if no suitable directory was found
if not best_match:
    print("Skipping this trait as no suitable data was found in TCGA.")


Available TCGA subdirectories: ['TCGA_Liver_Cancer_(LIHC)', 'TCGA_Lower_Grade_Glioma_(LGG)', 'TCGA_lower_grade_glioma_and_glioblastoma_(GBMLGG)', 'TCGA_Lung_Adenocarcinoma_(LUAD)', 'TCGA_Lung_Cancer_(LUNG)', 'TCGA_Lung_Squamous_Cell_Carcinoma_(LUSC)', 'TCGA_Melanoma_(SKCM)', 'TCGA_Mesothelioma_(MESO)', 'TCGA_Ocular_melanomas_(UVM)', 'TCGA_Ovarian_Cancer_(OV)', 'TCGA_Pancreatic_Cancer_(PAAD)', 'TCGA_Pheochromocytoma_Paraganglioma_(PCPG)', 'TCGA_Prostate_Cancer_(PRAD)', 'TCGA_Rectal_Cancer_(READ)', 'TCGA_Sarcoma_(SARC)', 'TCGA_Stomach_Cancer_(STAD)', 'TCGA_Testicular_Cancer_(TGCT)', 'TCGA_Thymoma_(THYM)', 'TCGA_Thyroid_Cancer_(THCA)', 'TCGA_Uterine_Carcinosarcoma_(UCS)', '.DS_Store', 'CrawlData.ipynb', 'TCGA_Acute_Myeloid_Leukemia_(LAML)', 'TCGA_Adrenocortical_Cancer_(ACC)', 'TCGA_Bile_Duct_Cancer_(CHOL)', 'TCGA_Bladder_Cancer_(BLCA)', 'TCGA_Breast_Cancer_(BRCA)', 'TCGA_Cervical_Cancer_(CESC)', 'TCGA_Colon_and_Rectal_Cancer_(COADREAD)', 'TCGA_Colon_Cancer_(COAD)', 'TCGA_Endometrioid_Canc


Clinical data columns:
['CDE_ID_3226963', '_INTEGRATION', '_PATIENT', '_cohort', '_primary_disease', '_primary_site', 'additional_pharmaceutical_therapy', 'additional_radiation_therapy', 'adenocarcinoma_invasion', 'age_at_initial_pathologic_diagnosis', 'alcohol_history_documented', 'alcoholic_exposure_category', 'amount_of_alcohol_consumption_per_day', 'anatomic_neoplasm_subdivision', 'anatomic_neoplasm_subdivision_other', 'bcr_followup_barcode', 'bcr_patient_barcode', 'bcr_sample_barcode', 'days_to_birth', 'days_to_collection', 'days_to_death', 'days_to_diabetes_onset', 'days_to_initial_pathologic_diagnosis', 'days_to_last_followup', 'days_to_new_tumor_event_additional_surgery_procedure', 'days_to_new_tumor_event_after_initial_treatment', 'days_to_pancreatitis_onset', 'family_history_of_cancer', 'followup_case_report_form_submission_reason', 'followup_treatment_success', 'form_completion_date', 'frequency_of_alcohol_consumption', 'gender', 'histologic_grading_tier_category', 'histolo

### Step 2: Find Candidate Demographic Features

In [3]:
# 1. Identify columns that likely contain information about patients' age and gender
candidate_age_cols = ['age_at_initial_pathologic_diagnosis', 'days_to_birth']
candidate_gender_cols = ['gender']

# 2. Extract and preview candidate columns from clinical data
# First, let's get the clinical data file path
cohort_dir = os.path.join(tcga_root_dir, 'TCGA_Stomach_Cancer_(STAD)')
clinical_file_path, _ = tcga_get_relevant_filepaths(cohort_dir)

# Load the clinical data
clinical_data = pd.read_csv(clinical_file_path, sep='\t', index_col=0)

# Extract age-related columns
age_columns = clinical_data[candidate_age_cols]
print("Age-related columns preview:")
age_preview = preview_df(age_columns, n=5)
print(age_preview)

# Extract gender-related columns
gender_columns = clinical_data[candidate_gender_cols]
print("\nGender-related columns preview:")
gender_preview = preview_df(gender_columns, n=5)
print(gender_preview)


Age-related columns preview:
{'age_at_initial_pathologic_diagnosis': [70.0, 51.0, 51.0, 62.0, 52.0], 'days_to_birth': [nan, nan, -18698.0, -22792.0, -19014.0]}

Gender-related columns preview:
{'gender': ['MALE', 'MALE', 'FEMALE', 'MALE', 'MALE']}


### Step 3: Select Demographic Features

In [4]:
# Examine the age-related columns
age_col = None
gender_col = None

# Age dictionary preview
age_dict = {'age_at_initial_pathologic_diagnosis': [70.0, 51.0, 51.0, 62.0, 52.0], 
            'days_to_birth': [float('nan'), float('nan'), -18698.0, -22792.0, -19014.0]}

# Gender dictionary preview
gender_dict = {'gender': ['MALE', 'MALE', 'FEMALE', 'MALE', 'MALE']}

# For age column selection
for col, values in age_dict.items():
    # Check if the values are reasonable and not mostly missing
    if not all(pd.isna(v) for v in values):
        age_col = col
        break

# For gender column selection
for col, values in gender_dict.items():
    # Check if the values make sense for gender
    valid_genders = ['MALE', 'FEMALE', 'male', 'female']
    if any(str(v).upper() in ['MALE', 'FEMALE'] for v in values if v is not None):
        gender_col = col
        break

# Print the chosen columns
print(f"Chosen age column: {age_col}")
print(f"Chosen gender column: {gender_col}")


Chosen age column: age_at_initial_pathologic_diagnosis
Chosen gender column: gender


### Step 4: Feature Engineering and Validation

In [5]:
# Step 1: Extract and standardize clinical features
# Create clinical features dataframe with trait (Canavan Disease) using patient IDs
clinical_features = tcga_select_clinical_features(
    clinical_df, 
    trait=trait, 
    age_col=age_col, 
    gender_col=gender_col
)

# Step 2: Normalize gene symbols in the gene expression data
# The gene symbols in TCGA genetic data are already standardized, but we'll normalize them for consistency
normalized_gene_df = normalize_gene_symbols_in_index(genetic_df)

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_df.to_csv(out_gene_data_file)
print(f"Normalized gene expression data saved to {out_gene_data_file}")
print(f"Gene expression data shape after normalization: {normalized_gene_df.shape}")

# Step 3: Link clinical and genetic data
# Transpose genetic data to have samples as rows and genes as columns
genetic_df_t = normalized_gene_df.T
# Save the clinical data for reference
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_features.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")
print(f"Clinical data shape: {clinical_features.shape}")

# Verify common indices between clinical and genetic data
clinical_indices = set(clinical_features.index)
genetic_indices = set(genetic_df_t.index)
common_indices = clinical_indices.intersection(genetic_indices)
print(f"Number of samples in clinical data: {len(clinical_indices)}")
print(f"Number of samples in genetic data: {len(genetic_indices)}")
print(f"Number of common samples: {len(common_indices)}")

# Link the data by using the common indices
linked_data = pd.concat([clinical_features.loc[list(common_indices)], genetic_df_t.loc[list(common_indices)]], axis=1)
print(f"Linked data shape: {linked_data.shape}")

# Step 4: Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, trait_col=trait)
print(f"Data shape after handling missing values: {linked_data.shape}")

# Step 5: Determine whether the trait and demographic features are severely biased
trait_biased, linked_data = judge_and_remove_biased_features(linked_data, trait=trait)

# Step 6: Conduct final quality validation and save information
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort="TCGA",
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=True,
    is_biased=trait_biased,
    df=linked_data,
    note=f"Dataset contains TCGA glioma and brain tumor samples with gene expression and clinical information for {trait}."
)

# Step 7: Save linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset deemed not usable based on validation criteria. Data not saved.")

print("Preprocessing completed.")

Normalized gene expression data saved to ../../output/preprocess/Lactose_Intolerance/gene_data/TCGA.csv
Gene expression data shape after normalization: (19848, 183)
Clinical data saved to ../../output/preprocess/Lactose_Intolerance/clinical_data/TCGA.csv
Clinical data shape: (196, 3)
Number of samples in clinical data: 196
Number of samples in genetic data: 183
Number of common samples: 183
Linked data shape: (183, 19851)


Data shape after handling missing values: (183, 19851)
For the feature 'Lactose_Intolerance', the least common label is '0' with 4 occurrences. This represents 2.19% of the dataset.
The distribution of the feature 'Lactose_Intolerance' in this dataset is severely biased.

Quartiles for 'Age':
  25%: 57.0
  50% (Median): 65.0
  75%: 73.0
Min: 35
Max: 88
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0' with 82 occurrences. This represents 44.81% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

Dataset deemed not usable based on validation criteria. Data not saved.
Preprocessing completed.
